In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_file = "F:/178/Tugas Akhir/dataset/clips_45sec_wav/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_file = "C:/Users/USER/Downloads/Tugas Akhir/dataset/clips_45sec_wav/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/AsikYesAjigile/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/AsikYesAjigile/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):

    train = clf.fit(data,values)
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,train,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def gridsearch(data,true):
    svr = SVR()
    Cs = [1,1e1,1e2,1e3,1e4,1e5,1e6]
    gammas = [1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6]
    parameters = {'C':Cs,'gamma':gammas}

    clf = GridSearchCV(svr, parameters, cv=10, scoring='r2')
    clf.fit(data,true)

    return clf.best_params_ , clf.best_score_

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

# CASE 1

In [4]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

(744, 26)


GRIDSEARCHCV

In [ ]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [ ]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

INITIAL MODEL

In [ ]:
svr_case1_a = SVR(kernel='rbf',C=1e1,gamma=1e-3,cache_size=500) #init Arousal
svr_case1_v = SVR(kernel='rbf',C=1,gamma=1e-3,cache_size=500) #init Valence

TRAIN

In [ ]:
# #Predict Arousal
a_a,train_a,df1_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
a_v,train_v,df1_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [ ]:
df1_a = df1_a.round(2)

display(df1_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
df1_v = df1_v.round(2)

display(df1_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(svr_case1_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case1_v,data,valence_mean) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
# save_predict(df1_a.round(2),df1_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 2

In [ ]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

GRIDSEARCH

In [ ]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [ ]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

INITIAL MODEL

In [ ]:
svr_case2_a = SVR(kernel='rbf',C=1e3,gamma=1e-2,cache_size=500) #init Arousal
svr_case2_v = SVR(kernel='rbf',C=1e3,gamma=1e-2,cache_size=500) #init Valence

TRAIN

In [ ]:
# #Predict Arousal
a_a,train_a,df2_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case2_a)
# #Predict Valence
a_v,train_v,df2_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case2_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [ ]:
df2_a = df2_a.round(2)

display(df2_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
df2_v = df2_v.round(2)

display(df2_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(svr_case2_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case2_v,data,valence_mean) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
save_predict(df2_a.round(2),df2_v.round(2),datafitur)
save_model(train_a,datafitur,'arousal/') #save model Arousal
save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 3

In [ ]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

GRIDSEARCH

In [ ]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [ ]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

INITAL MODEL

In [ ]:
svr_case3_a = SVR(kernel='rbf',C=1e3,gamma=1e-2,cache_size=500) #init Arousal
svr_case3_v = SVR(kernel='rbf',C=1e1,gamma=1e-1,cache_size=500) #init Valence

TRAIN

In [ ]:
# #Predict Arousal
a_a,train_a,df3_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case3_a)
# #Predict Valence
a_v,train_v,df3_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case3_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [ ]:
df3_a = df3_a.round(2)

display(df3_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(svr_case3_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case3_v,data,valence_mean) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
save_predict(df3_a.round(2),df3_v.round(2),datafitur)
save_model(train_a,datafitur,'arousal/') #save model Arousal
save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 4

In [ ]:
datafitur =  "case4" #CASE 4
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,arousal_mean) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,valence_mean) #split valence

GRIDSEAERCH

In [ ]:
param_a, score_a = gridsearch(data,arousal_mean) #Gridsearch Arousal
param_v, score_v = gridsearch(data,valence_mean) #Grinsearch Valence

In [ ]:
print("parameter Arousal:\t",param_a)
print("parameter Valence:\t",param_v)

INITIAL MODEL

In [ ]:
svr_case4_a = SVR(kernel='rbf',C=1e2,gamma=1e-2,cache_size=500) #init Arousal
svr_case4_v = SVR(kernel='rbf',C=1e2,gamma=1e-2,cache_size=500) #init Valence

TRAIN

In [ ]:
# #Predict Arousal
a_a,train_a,df4_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case4_a)
# #Predict Valence
a_v,train_v,df4_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case4_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

AROUSAL

In [ ]:
df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

VALENCE

In [ ]:
df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

REGRESSION

In [ ]:
scores_a = regresion(svr_case4_a,data,arousal_mean) #Arousal Regression
scores_v = regresion(svr_case4_v,data,valence_mean) #Arousal Regression

AROUSAL

In [ ]:
print("R2 Score Valence:\t",scores_a)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [ ]:
print("R2 Score Valence:\t",scores_v)
print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [ ]:
# save_predict(df4_a.round(2),df4_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence